# L3: Multi-agent Customer Support Automation

In this lesson, you will learn about the six key elements which help make Agents perform even better:
- Role Playing
- Focus
- Tools
- Cooperation
- Guardrails
- Memory

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [15]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, API and LLM

In [16]:
from crewai import Agent, Task, Crew

In [ ]:
import os
from google.colab import userdata

openai_api_key = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'

In [37]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'

## Role Playing, Focus and Cooperation

In [38]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at Niva Bupa Insurance Agency and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True
)

- By not setting `allow_delegation=False`, `allow_delegation` takes its default value of being `True`.
- This means the agent _can_ delegate its work to another agent which is better suited to do a particular task.

In [39]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at Niva Bupa Insurance Agency and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True
)

* **Role Playing**: Both agents have been given a role, goal and backstory.
* **Focus**: Both agents have been prompted to get into the character of the roles they are playing.
* **Cooperation**: Support Quality Assurance Agent can delegate work back to the Support Agent, allowing for these agents to work together.

## Tools, Guardrails and Memory

### Tools

- Import CrewAI tools

In [22]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

### Possible Custom Tools
- Load customer data
- Tap into previous conversations
- Load data from a CRM
- Checking existing bug reports
- Checking existing feature requests
- Checking ongoing tickets
- ... and more

- Some ways of using CrewAI tools.

```Python
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
```

- Instantiate a document scraper tool.
- The tool will scrape a page (only 1 URL) of the CrewAI documentation.

In [23]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://www.nivabupa.com/travel-insurance.html"
)

##### Different Ways to Give Agents Tools

- Agent Level: The Agent can use the Tool(s) on any Task it performs.
- Task Level: The Agent will only use the Tool(s) when performing that specific Task.

**Note**: Task Tools override the Agent Tools.

### Creating Tasks
- You are passing the Tool on the Task Level.

In [40]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

- `quality_assurance_review` is not using any Tool(s)
- Here the QA Agent will only review the work of the Support Agent

In [41]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


### Creating the Crew

#### Memory
- Setting `memory=True` when putting the crew together enables Memory.

In [42]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=True,
  memory=True
)

2025-02-13 08:04:02,463 - 7955664896 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


### Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

#### Guardrails
- By running the execution below, you can see that the agents and the responses are within the scope of what we expect from them.

In [43]:
inputs = {
    "customer": "Vettura.ai",
    "person": "Ashish Sahani",
    "inquiry": "Hey I am travelling to Japan for a conference"
    "from 22nd to 27 Februray. How can I get a travel insurance? Can I cancel if my trip gets cancelled?"
}
result = crew.kickoff(inputs=inputs)

# Agent: Senior Support Representative
## Task: Vettura.ai just reached out with a super important ask:
Hey I am travelling to Japan for a conferencefrom 22nd to 27 Februray. How can I get a travel insurance? Can I cancel if my trip gets cancelled?

Ashish Sahani from Vettura.ai is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


# Agent: Senior Support Representative
## Thought: I need to gather detailed information about the travel insurance options available through Niva Bupa, including how to obtain it and the specifics related to trip cancellation.
## Using tool: Read website content
## Tool Input: 
"{}"
## Tool Output: 
Buy Travel Insurance Plans Online in India 2024 | Niva Bupa × Hospital Network Live Well Chat with Us Become An Agent For a New Policy 1800-309-3333 Customer Helpline 1860-500-8888 Become An Agent Claim Renew Health Insurance 

- Display the final result as Markdown.

In [44]:
from IPython.display import Markdown
Markdown(result.raw)

Hi Ashish,

Thank you for reaching out! I'm thrilled to assist you with your travel insurance needs for your upcoming trip to Japan from February 22nd to 27th. Let's ensure you're fully equipped with all the information you need for a worry-free experience.

### How to Get Travel Insurance:
Purchasing a travel insurance policy is incredibly straightforward through our Niva Bupa website. Here’s a step-by-step guide to help you out:

1. **Visit the Niva Bupa Travel Insurance page.** You can find it easily on our main website, typically under the insurance sections.
2. **Enter your travel details:** Input your travel dates (22nd - 27th February), along with your personal information, and select coverage options that suit your travel plans.
3. **Receive a quote:** After submitting your details, you'll receive a personalized quote tailored to your requirements. This quote allows you to review various policy options and find one that fits your needs.
4. **Complete the purchase:** Once you've chosen your preferred policy, simply fill in the necessary information to finalize the purchase. You'll then be covered for the entirety of your trip!

### Trip Cancellation Policy:
Regarding your question about trip cancellation, I’m pleased to confirm that our travel insurance covers financial losses due to trip cancellations occurring before your policy's effective start date. It's a smart idea to acquire travel insurance as soon as your trip is booked, giving you protection from the moment you finalize plans.

If you need to cancel your trip after your insurance has taken effect, I'd suggest reviewing the policy document closely. It details specific conditions for claims related to trip cancellations. Generally, expenses resulting from trip interruptions, delays, or cancellations due to unforeseen events will be reimbursed, ensuring that you have peace of mind during your travels.

### Additional Benefits:
With our travel insurance, you're not only protected against cancellations but also enjoy a host of other benefits like:

- **Emergency Medical Expenses:** Get coverage for unplanned medical emergencies during your trip.
- **Baggage Coverage:** Protect against loss or delay of baggage, so your belongings are safe.
- **Missed Flight Assistance:** If a missed flight throws a wrench in your plans, we’re here to help.
- **Much More!** These coverages ensure you are well-prepared for any unexpected situations that may arise.

If you have any further questions or need clarification on anything related to your coverage or the claims process, please don't hesitate to reach out! I'm here to help make your travel as smooth and enjoyable as possible.

Wishing you safe travels and an incredible time at your conference in Japan!

Best regards,  
[Your Name]  
Senior Support Representative  
Niva Bupa